In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython


In [2]:
%reload_ext Cython

In [3]:
import sys
sys.path.append('..')
import warnings
def warn(message = None, category=None, stacklevel=1, source=None):
    pass

warnings.warn = warn

In [35]:
import scipy.stats as sts
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
#custom
import data_process, path
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

### Load Pmpm and IDs

In [5]:
%%time
ids_csv = '../../../data/test_id.csv'
pmpm_p = '../../../data/PMPM.p'

treated_ids = pd.read_csv(ids_csv)
pmpm = pd.read_pickle(pmpm_p)

#drop pmpm duplicates
pmpm.drop_duplicates(subset = ['PERS_ID', 'MYR'], inplace = True)


CPU times: user 8.75 s, sys: 10.8 s, total: 19.5 s
Wall time: 19.5 s


### Drop Unecessary data points

In [6]:
pmpm = pmpm[pmpm.MYR >= 202103].reset_index(drop = True)

In [7]:
pmpm.head()

PERS_ID     MYR MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT MEDICAID_LOB  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0      -3  202103        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
1      -3  202104        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
2      -3  202105        M              121          0   Commercial       0.0       0.0      0.0           0.0                0.0     0.0        0.0                  0.0           0.0        0.0            0.0        0.0         0.0     0.0               0.0          0.0                0.0                0.0                   0.0      0.0                 0.0            0.0                      0.0                0.0          0.0                0          0                    0.0          0.0              0.0          0.0                0.0               0.0             0.0                   0.0                     0.0                   0.0                     0.0                         0.0             0.0             0.0               0.0              0.0                0.0              0.0             0.0           0.0          0.0             0.0           0.0       0.0             0.0          0.0                 0.0                    0.0                    0.0         0.0   
3      -3  202106 

In [8]:
pmpm.MBR_GNDR.replace(to_replace = {'F':0, 'M': 1}, inplace = True)
# pmpm.MEDICAID_LOB.unique()

### Include Line of Bussiness Feature 1 Not medicaid 0 medicaid

In [9]:
# medicaid_mask = pmpm.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
# pmpm['LOB'] = 1
# pmpm.loc[medicaid_mask, 'LOB'] = 0

### Separate Treated Controls

In [10]:
treated_ids.iloc[0]

PERS_ID    2778075
Name: 0, dtype: int64

In [11]:
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)

In [12]:
%%time
id_filter = pmpm.PERS_ID.isin(list(treated_ids.values[:,0]))
treated, controls = pmpm[id_filter].reset_index(drop = True), pmpm[~id_filter].reset_index(drop = True)

CPU times: user 415 ms, sys: 347 ms, total: 762 ms
Wall time: 762 ms


In [13]:
treated = treated[treated.MYR >= 202108].reset_index(drop = True)

In [14]:
#cols = list(col for col in treated2.columns if col not in ['PERS_ID', 'MYR'])
#columns_to_match = cols
columns_to_match = ['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'MBR_GNDR','CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT', 'ACUTE', 'ACUTE2']

In [15]:
def get_data(data_struct, cont):
    cont = cont.copy()
    keys = list(data_struct.keys())
    dat1 = pd.DataFrame()
    for key in keys:
        dat1 = pd.concat([dat1, data_struct[key]], ignore_index = True)
    dat2 = dat1[['PERS_ID', 'MYR']]
    dat = pd.merge(dat2, cont, on = ['PERS_ID', 'MYR']).reset_index(drop = True)
    return dat

### General Configurations for All Matching Scenaria

In [16]:
#columns_to_match = columns_to_match + ['IP_BIN2', 'ER_BIN2']
#hier_cols = ['IP_BIN2', 'ER_BIN2', 'ACUTE2', 'ACUTE6', 'IP_BIN6', 'ER_BIN6']
hier_cols = None #['CC_DX_CNT']
hier_range = [1]
exact_match_dict = {'ACUTE':None, 'ACUTE2':None}
match_on = [ 'ALLOW_AMT', 'CC_TOBACCO', 'CC_PRESSURE', 'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_DX_CNT']
            


In [17]:
cols_to_keep = ["PERS_ID", "MYR"]

## Load Model

In [18]:
import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

model = models.load_model("../../../temp/TF_Latent_Model_m6_Canary_stacked", compile=False)

/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib64/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type

In [19]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [20]:
%%time
#transform data of sick and healthy treated, and controls
tr2 = data_process.transform_features_with_NN(model, treated[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
cntrls = data_process.transform_features_with_NN(model, controls[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

CPU times: user 11.6 s, sys: 3.47 s, total: 15 s
Wall time: 8.62 s


## Scenario 1 Rigorous

In [21]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat1 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 42s, sys: 1min 30s, total: 3min 13s
Wall time: 15.3 s


In [22]:
dat1.shape

## Scenario 2 Non Rigorous

In [23]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat2 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 40s, sys: 1min 27s, total: 3min 7s
Wall time: 16.4 s


In [24]:
dat2.shape

### Scenario 3 Rigorous 6 months

In [36]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat3 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 57s, sys: 1min 37s, total: 3min 35s
Wall time: 35.3 s


In [37]:
dat3.MYR.min(), dat3.MYR.max()

(202103, 202108)

In [38]:
dat3.shape

## Scenario 4 Non Rigorous 6 Months

In [39]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrls.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat4 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:220845 before they were:220845
CPU times: user 1min 58s, sys: 1min 38s, total: 3min 36s
Wall time: 36.2 s


In [40]:
dat4.shape

## Scenario 5 Medicaid Rigorous 

In [41]:
controlsM = controls[controls.LOB == 0].reset_index(drop = True)

In [42]:
cntrlsM = data_process.transform_features_with_NN(model, controlsM[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)

In [43]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat5 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 39.7 s, sys: 33.9 s, total: 1min 13s
Wall time: 5.67 s


In [44]:
dat5.shape

## Scenario 6 Medicaid Non Rigorous

In [45]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':True}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat6 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 39 s, sys: 34 s, total: 1min 12s
Wall time: 5.66 s


In [46]:
dat6.shape

### Scenario 7 Medicaid Rigorous 6 months

In [47]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 5,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat7 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 1min 29s, sys: 1min 18s, total: 2min 47s
Wall time: 11.6 s


In [48]:
dat7.MYR.min(), dat7.MYR.max()

(202103, 202108)

In [49]:
dat7.shape

## Scenario 8 Medicaid Non Rigorous 6 Months

In [50]:
%%time
nnr_func_args = {'match_on':latent_cols, 'hier_cols':None,
                        'hier_range':None, 'neighbors_per_treated': 20,
                 'onset': "CC_DIABETES", "onset_val" : 0, 'exact_match': False, 'exact_match_dict': None, 'restrict_time':False}

rig_st, matched_data0 = matched_population(tr2.copy(), 
                                           cntrlsM.copy(),
                                           nearest_neighbor_func, 
                                           nnr_func_args)


dat8 = get_data(rig_st, controls)

Mathcing Function: nearest_neighbor_func
After filtering control ids are:53312 before they were:53312
CPU times: user 1min 34s, sys: 1min 22s, total: 2min 56s
Wall time: 12.2 s


In [51]:
dat8.shape

# Save Stats

In [52]:
from utils import statistics_first_month
import scipy.stats as sts

In [53]:
columns_to_match = treated.columns.tolist()

In [54]:
%%time
tr_desc0, Ntr, treated_first_month = statistics_first_month(treated, columns_to_match, return_array = True)
tr_desc0 = tr_desc0.T

con1, N1, controls_first_month = statistics_first_month(dat1, columns_to_match, return_array = True)
con1 = con1.T

con2, N2, controls_first_month = statistics_first_month(dat2, columns_to_match, return_array = True)
con2 = con2.T

con3, N3, controls_first_month = statistics_first_month(dat3, columns_to_match, return_array = True)
con3 = con3.T

con4, N4, controls_first_month = statistics_first_month(dat4, columns_to_match, return_array = True)
con4 = con4.T

con5, N5, controls_first_month = statistics_first_month(dat5, columns_to_match, return_array = True)
con5 = con5.T

con6, N6, controls_first_month = statistics_first_month(dat6, columns_to_match, return_array = True)
con6 = con6.T

con7, N7, controls_first_month = statistics_first_month(dat7, columns_to_match, return_array = True)
con7 = con7.T

con8, N8, controls_first_month = statistics_first_month(dat8, columns_to_match, return_array = True)
con8 = con8.T

un_con_all, N9, controls_first_month =  statistics_first_month(controls, columns_to_match, return_array = True)
un_con_all = un_con_all.T

un_con_med, N10, controls_first_month =  statistics_first_month(controlsM, columns_to_match, return_array = True)
un_con_med = un_con_med.T



CPU times: user 1min 50s, sys: 1.8 s, total: 1min 51s
Wall time: 1min 51s


In [55]:
alli = pd.concat((tr_desc0[['mean','std']], un_con_all[['mean','std']], con1[['mean','std']], con2[['mean','std']], con3[['mean','std']], 
                  con4[['mean','std']], un_con_med[['mean','std']], con5[['mean','std']], con6[['mean','std']], con7[['mean','std']], con8[['mean','std']]),
                 axis = 1, keys = ['Treated', 'Unmatched Controls', 'Rig', 'NonRig', 'Rig6', 'NonRig6', 'Unmatched Med_Controls', 
                                   'Rig-Med', 'NonRig-Med', 'Rig-Med6','NonRig-Med6'])

In [56]:
pd.options.display.float_format = '{:.2f}'.format

In [67]:
alli[2:]

Treated         Unmatched Controls            Rig         NonRig           Rig6         NonRig6         Unmatched Med_Controls         Rig-Med         NonRig-Med         Rig-Med6         NonRig-Med6        
                              mean     std               mean     std   mean     std   mean     std   mean     std    mean     std                   mean     std    mean     std       mean     std     mean     std        mean     std
MBR_GNDR                      0.34    0.48               0.44    0.50   0.34    0.47   0.34    0.47   0.36    0.48    0.34    0.47                   0.40    0.49    0.31    0.46       0.32    0.47     0.31    0.46        0.31    0.46
AGE_AT_MIDMONTH              37.87   11.87              48.09   20.25  39.85   12.83  41.19   13.97  39.48   13.54   39.79   13.66                  34.04   16.91   39.34   12.44      40.38   13.33    38.66   12.12       38.69   12.41
ALLOW_AMT                   709.13 1425.41            1030.41 5321.78 770.71 1722.07 752.53 1757.82 761.03 1525.27  788.40 1658.48                 822.60 2931.48  771.65 1775.28     741.42 1759.16   770.53 1594.11      799.29 1716.56
ALLOW_ER                     22.19  110.86              20.08  192.30  19.94  132.13  17.05  121.57  13.26  146.02   15.25  124.48                  23.18  140.56   28.04  121.30      22.30  116.69    10.34   65.79       19.74  101.91
ALLOW_IP                    279.28 1205.83             249.13 4264.12  51.17  560.29  34.54  465.00 157.71  975.35   89.98  723.53                 192.86 2071.91   72.25  582.77     104.37  761.82   168.44 1008.15      119.26  824.14
CC_ADHD                       0.23    0.42               0.04    0.20   0.07    0.26   0.08    0.28   0.07    0.26    0.08    0.27                   0.09    0.28    0.10    0.30       0.11    0.32     0.13    0.34        0.11    0.31
CC_ALZHEIMER                  0.01    0.09               0.01    0.07   0.00    0.00   0.00    0.00   0.00    0.04    0.00    0.03                   0.00    0.03    0.00    0.00       0.00    0.00     0.00    0.00        0.00    0.03
CC_ALZHEIMER_PLUS             0.04    0.19               0.02    0.13   0.00    0.06   0.01    0.09   0.01    0.09    0.01    0.08                   0.01    0.08    0.01    0.10       0.01    0.11     0.01    0.10        0.01    0.11
CC_AMI                        0.02    0.13               0.00    0.06   0.00    0.00   0.00    0.04   0.01    0.09    0.00    0.05                   0.00    0.05    0.00    0.04       0.00    0.07     0.01    0.10        0.00    0.06
CC_ANEMIA                     0.16    0.37               0.07    0.26   0.09    0.29   0.09    0.28   0.08    0.27    0.08    0.27                   0.07    0.26    0.12    0.32       0.12    0.32     0.11    0.31        0.12    0.32
CC_ANXIETY_DISORDER           0.90    0.30               0.22    0.42   0.39    0.49   0.39    0.49   0.37    0.48    0.38    0.49                   0.29    0.46    0.50    0.50       0.47    0.50     0.50    0.50        0.46    0.50
CC_ARTHRITIS                  0.14    0.35               0.14    0.34   0.12    0.32   0.13    0.33   0.09    0.29    0.10    0.31                   0.08    0.27    0.17    0.38       0.16    0.37     0.13    0.34        0.13    0.33
CC_ASTHMA                     0.21    0.41               0.06    0.23   0.10    0.29   0.09    0.28   0.10    0.30    0.10    0.30                   0.09    0.28    0.13    0.34       0.14    0.34     0.15    0.35        0.13    0.33
CC_ATRIAL_FIB                 0.04    0.19               0.03    0.17   0.01    0.10   0.02    0.13   0.02    0.13    0.01    0.12                   0.01    0.09    0.01    0.12       0.01    0.11     0.01    0.11        0.01    0.09
CC_AUTISM                     0.04    0.19               0.01    0.08   0.01    0.08   0.01    0.08   0.01    0.11    0.01    0.10                   0.02    0.13    0.01    0.11       0.01    0.09     0.01    0.11        0.01    0.10
CC_BIPOLAR                    0.41    0.49       

In [68]:
alli[2:].to_csv("./results/stats_mean_std/seq_9_16_mean_std.csv", header=True, index=True)

## Latent Stats

## Save latent features

In [66]:
dat_lists = [dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8]
file_id_start = 9
for data in dat_lists:
    dat_lat = data_process.transform_features_with_NN(model, data[path.col_seq_can_all].copy(), cols_to_keep, latent_dim=16)
    dat_lat.to_csv(f"./results/latent_features/seq_{file_id_start}.csv", index=False, header=True)
    file_id_start += 1

## Save IDs

In [69]:
ids = pd.concat([dat1.PERS_ID, dat2.PERS_ID, dat3.PERS_ID, dat4.PERS_ID, dat5.PERS_ID, dat6.PERS_ID, dat7.PERS_ID, dat8.PERS_ID], axis=1, ignore_index=True)
ids.rename(columns={0:"seq9", 1:"seq10", 2:"seq11", 3:"seq12", 4:"seq13", 5:"seq14", 6:"seq15", 7:"seq16"}, inplace=True)
ids.fillna(-1, inplace=True)
ids = ids.astype(int)

In [70]:
ids

seq9    seq10    seq11    seq12    seq13    seq14    seq15    seq16
0     2779836  2779836  2775576  2775576  5036526  5036526  2809522  2809522
1     5036619  5036619  4079774  4079774  2837861  2837861  5037421  5037421
2     5036526  5036526  5045112  5045112  2950124  2950124  2821732  2821732
3     2837861  2837861  2809522  2809522  3708646  3708646  2789472  2789472
4     3938421  3938421  2796878  2796878  2958127  2958127  2872508  2872508
...       ...      ...      ...      ...      ...      ...      ...      ...
2215       -1  5038782       -1  2946009       -1  2867113       -1  2946040
2216       -1  4154745       -1  2976491       -1  4367820       -1  2958127
2217       -1  3738730       -1  4332656       -1  4023921       -1  2982693
2218       -1  2956519       -1  2966935       -1  2829682       -1  2966875
2219       -1  2882530       -1  4232299       -1  4182390       -1  3710628

[2220 rows x 8 columns]

In [71]:
ids.to_csv("./results/ids/ids_seq_9_16.csv", index=True, header=True)

## Ratios with/without Std

In [104]:
disease_cols = [cols for cols in treated.columns if "CC" in cols and cols!='CC_DX_CNT']
other_binary_cols = ['MBR_GNDR', 'LOB', 'PREGNANCY', 'IP_BIN', 'ER_BIN', 'ACUTE']
binary_cols = other_binary_cols + disease_cols 
restricted_cols = ['PERS_ID', 'MYR', 'MEDICAID_LOB']
continuous_cols = [cols for cols in treated.columns if cols not in binary_cols + restricted_cols]

In [105]:
def calculate_continuous_d(t, c, std):
    Xt = np.mean(t)
    Xc = np.mean(c)
    St = np.std(t)
    Sc = np.std(c)
    if std==1: 
        d = np.abs((Xt-Xc)/(np.sqrt((St**2 + Sc**2)/2)))
    else:
        d = np.abs((Xt-Xc))
    return d

In [106]:
def calculate_binary_d(t, c, std):
    Pt = np.mean(t)
    Pc = np.mean(c)
    if std==1:
        d = np.abs((Pt-Pc)/(np.sqrt((Pt*(1-Pt)+Pc*(1-Pc))/2)))
    else:
        d = np.abs(Pt-Pc)
    return d

### 1.1 Whole Population as controls

In [107]:
std = 1

In [108]:
d_metric_before_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controls[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controls[[col]].values, std=std)
        d_metric_before_all.loc[0,col] = d

In [109]:
d_metric_after_all = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[:4]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_all.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [110]:
d_metric_after_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.00             0.16       0.04      0.02      0.24     0.45          0.13               0.23    0.19       0.21                 1.26          0.08       0.32           0.17       0.20        0.79    0.02              0.00         0.21               0.09               0.40                  0.07     0.44                0.09           1.01                     0.46               0.13         0.13             0.32       1.22                   0.13         0.25             0.46         0.08               0.23              0.34             nan                  0.06                    0.06                  0.06                    0.14                        0.21             nan             nan              0.20              nan               0.03             0.28            0.09          0.10         0.17            0.16          0.06      0.46             nan         0.03                0.19                   0.15                   0.09        0.02             0.07           0.02   
1      0.01             0.26       0.02      0.05      0.27     0.40          0.13               0.20    0.16       0.23                 1.26          0.05       0.34           0.10       0.21        0.81    0.06              0.01         0.22               0.05               0.39                  0.08     0.39                0.06           1.08                     0.39               0.12         0.09             0.30       1.16                   0.10         0.23             0.43         0.07               0.21              0.30             nan                  0.03                    0.03                  0.03                    0.14                        0.18             nan             nan              0.19              nan               0.00             0.19            0.00          0.10         0.10            0.15          0.10      0.39            0.10         0.02                0.14                   0.17                   0.05        0.05             0.03           0.08   
2      0.03             0.11       0.02      0.07      0.12     0.40          0.10               0.20    0.10       0.26                 1.30          0.17       0.30           0.12       0.17        0.79    0.05              0.07         0.16               0.09               0.45                  0.07     0.45                0.06           1.04                     0.41               0.13         0.20             0.39       1.23                   0.10         0.21             0.54         0.11               0.16              0.31             nan                   nan                     nan                   nan                    0.12                        0.21             nan             nan              0.22              nan               0.02             0.17            0.03          0.12         0.18            0.16          0.12      0.43            0.06         0.04                0.17                   0.06                    nan        0.00             0.02           0.08   
3  

In [111]:
d_metric_before_all

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.19             0.70       0.04      0.05      0.04     0.56          0.04               0.11    0.14       0.27                 1.82          0.01       0.45           0.01       0.21        1.01    0.16              0.08         0.39               0.04               0.42                  0.06     0.34                0.07           1.47                     0.45               0.10         0.12             0.32       1.23                   0.10         0.27             0.57         0.15               0.28              0.23            0.01                  0.03                    0.04                  0.01                    0.15                        0.21            0.01            0.00              0.22             0.06               0.20             0.02            0.01          0.15         0.07            0.15          0.12      0.41            0.09         0.18                0.15                   0.10                   0.03        0.11             0.10           0.21   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.63         0.16                0.15     1.25              0.44                    0.53            0.16             0.05       0.04        1.23                0.09              0.05 2.58       0.04    0.29    0.22   0.35  1.11  1.13     1.31     1.03     0.88    1.29     0.59     0.56    0.77

In [112]:
ratio_1_4 = d_metric_after_all.values / d_metric_before_all.values
ratio_1_4 = pd.DataFrame(ratio_1_4, columns=d_metric_after_all.columns)

In [113]:
ratio_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.23       1.04      0.36      5.58     0.80          3.68               2.21    1.35       0.78                 0.69          6.13       0.70          13.42       0.96        0.78    0.12              0.00         0.55               2.03               0.94                  1.11     1.31                1.18           0.69                     1.00               1.34         1.07             1.00       0.99                   1.38         0.94             0.80         0.52               0.82              1.45             nan                  2.25                    1.63                  4.36                    0.93                        1.01             nan             nan              0.91              nan               0.14            11.57            9.27          0.66         2.51            1.13          0.50      1.11             nan         0.17                1.30                   1.49                   3.29        0.14             0.66           0.09   
1      0.08             0.38       0.65      0.89      6.17     0.71          3.68               1.88    1.15       0.85                 0.69          3.61       0.75           8.41       1.01        0.80    0.39              0.12         0.56               1.24               0.92                  1.22     1.15                0.83           0.74                     0.86               1.15         0.80             0.94       0.94                   1.01         0.84             0.75         0.46               0.77              1.31             nan                  1.12                    0.82                  2.18                    0.93                        0.84             nan             nan              0.88              nan               0.02             7.82            0.18          0.69         1.47            1.04          0.84      0.95            1.06         0.10                0.95                   1.72                   2.01        0.51             0.25           0.41   
2      0.16             0.16       0.55      1.38      2.67     0.72          2.69               1.88    0.68       0.99                 0.71         12.87       0.66           9.98       0.81        0.78    0.32              0.86         0.41               2.03               1.08                  1.11     1.34                0.83           0.71                     0.90               1.34         1.66             1.20       1.00                   1.01         0.77             0.95         0.70               0.58              1.33             nan                   nan                     nan                   nan                    0.79                        1.01             nan             nan              0.99              nan               0.09             7.23            3.48          0.79         2.62            1.13          1.01      1.05            0.67         0.19                1.17                   0.61                    nan        0.04             0.20           0.38   
3  

In [114]:
mask_1_4 = ratio_1_4[ratio_1_4<=1.0]

In [115]:
mask_1_4

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.23        nan      0.36       nan     0.80           nan                nan     nan       0.78                 0.69           nan       0.70            nan       0.96        0.78    0.12              0.00         0.55                nan               0.94                   nan      nan                 nan           0.69                      nan                nan          nan             1.00       0.99                    nan         0.94             0.80         0.52               0.82               nan             nan                   nan                     nan                   nan                    0.93                         nan             nan             nan              0.91              nan               0.14              nan             nan          0.66          nan             nan          0.50       nan             nan         0.17                 nan                    nan                    nan        0.14             0.66           0.09   
1      0.08             0.38       0.65      0.89       nan     0.71           nan                nan     nan       0.85                 0.69           nan       0.75            nan        nan        0.80    0.39              0.12         0.56                nan               0.92                   nan      nan                0.83           0.74                     0.86                nan         0.80             0.94       0.94                    nan         0.84             0.75         0.46               0.77               nan             nan                   nan                    0.82                   nan                    0.93                        0.84             nan             nan              0.88              nan               0.02              nan            0.18          0.69          nan             nan          0.84      0.95             nan         0.10                0.95                    nan                    nan        0.51             0.25           0.41   
2      0.16             0.16       0.55       nan       nan     0.72           nan                nan    0.68       0.99                 0.71           nan       0.66            nan       0.81        0.78    0.32              0.86         0.41                nan                nan                   nan      nan                0.83           0.71                     0.90                nan          nan              nan       1.00                    nan         0.77             0.95         0.70               0.58               nan             nan                   nan                     nan                   nan                    0.79                         nan             nan             nan              0.99              nan               0.09              nan             nan          0.79          nan             nan           nan       nan            0.67         0.19                 nan                   0.61                    nan        0.04             0.20           0.38   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [116]:
mask_1_4.columns[~mask_1_4.isnull().any()].size

In [117]:
mask_1_4.columns[~mask_1_4.isnull().any()]

Index(['MBR_GNDR', 'AGE_AT_MIDMONTH', 'CC_ADHD', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_BPH', 'CC_BREAST_CANCER', 'CC_CATARACT', 'CC_DEPRESSION', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_GEN', 'CC_HYPERLIPIDEMIA', 'CC_INT_DISAB', 'CC_MIGRAINE', 'CC_OBESITY', 'CC_OSTEOPOROSIS', 'CC_PERIPHERAL', 'CC_PERSONALITY', 'CC_PRESSURE', 'CC_PROSTATE_CANCER', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_TOBACCO', 'LOB', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [118]:
mask_1_4.columns[mask_1_4.isnull().all()].size

In [119]:
mask_1_4.columns[mask_1_4.isnull().all()]

Index(['ALLOW_IP', 'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_ARTHRITIS', 'CC_ATRIAL_FIB', 'CC_CEREBRAL_PALSY', 'CC_COLORECTAL_CANCER', 'CC_COPD', 'CC_DEVELOP_DELAYS', 'CC_ENDOMETRIAL_CANCER', 'CC_HEART_FAILURE', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_HYPERTENSION', 'CC_ISCHEMIC', 'CC_MUSCULAR_DYSTROPHY', 'CC_SPINAL_CORD', 'CC_STROKE', 'PREGNANCY'], dtype='object')

### 1.2 Only Medicaid Controls

In [120]:
d_metric_before_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        if col in binary_cols:
            d = calculate_binary_d(treated[[col]].values, controlsM[[col]].values, std=std)
        elif col in continuous_cols:
            d = calculate_continuous_d(treated[[col]].values, controlsM[[col]].values, std=std)
        d_metric_before_med.loc[0,col] = d

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [121]:
d_metric_before_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.10             0.20       0.02      0.04      0.09     0.38          0.12               0.19    0.15       0.26                 1.49          0.17       0.33           0.18       0.12        0.81    0.01              0.04         0.20               0.05               0.52                  0.10     0.34                0.08           1.18                     0.40               0.04         0.22             0.27       1.30                   0.12         0.21             0.54         0.03               0.18              0.34            0.02                  0.04                    0.05                  0.02                    0.09                        0.17            0.01            0.01              0.15             0.04               0.13             0.28            0.11          0.09         0.18            0.08          0.08      0.42            0.06         0.12                0.15                   0.09                   0.03        0.12             0.05           0.07   

   CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER  CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  LOB  PREGNANCY  IP_BIN  ER_BIN  ACUTE  IP_C  ER_C  ACUTE_C  IP_BIN6  ER_BIN6  ACUTE6  IP_BIN2  ER_BIN2  ACUTE2  
0            0.56         0.14                0.06     1.06              0.35                    0.42            0.17             0.07       0.08        0.89                0.08              0.05  nan       0.05    0.26    0.09   0.23  1.05  0.81     1.04     0.97     0.67    1.07     0.54     0.41    0.61

In [122]:
d_metric_after_med = pd.DataFrame()
for col in treated.columns.tolist():
    if col not in restricted_cols:
        i=0
        for dat_ in dat_lists[4:]:
            if col in binary_cols:
                d = calculate_binary_d(treated[[col]].values, dat_[[col]].values, std=std)
            elif col in continuous_cols:
                d = calculate_continuous_d(treated[[col]].values, dat_[[col]].values, std=std)
            d_metric_after_med.loc[i,col] = d
            i+=1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [123]:
d_metric_after_med

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.07             0.13       0.04      0.05      0.22     0.35          0.13               0.17    0.16       0.13                 0.98          0.07       0.20           0.12       0.15        0.59    0.04              0.00         0.17               0.09               0.32                  0.10     0.26                0.06           0.82                     0.32               0.07         0.02             0.18       0.84                   0.13         0.14             0.28         0.08               0.20              0.29             nan                  0.06                    0.06                  0.06                    0.04                        0.07             nan             nan              0.04             0.06               0.01             0.18            0.02          0.10         0.07            0.14          0.12      0.38            0.09         0.00                0.09                   0.18                   0.09        0.08             0.02           0.08   
1      0.03             0.24       0.01      0.00      0.18     0.31          0.13               0.16    0.13       0.13                 1.05          0.05       0.18           0.16       0.20        0.65    0.02              0.00         0.23               0.06               0.31                  0.08     0.25                0.08           0.84                     0.34               0.08         0.00             0.24       0.81                   0.10         0.16             0.32         0.05               0.21              0.23            0.03                  0.04                    0.04                  0.04                    0.04                        0.09             nan             nan              0.07             0.04               0.02             0.10            0.01          0.07         0.03            0.16          0.14      0.32            0.07         0.03                0.10                   0.13                   0.04        0.09             0.03           0.14   
2      0.08             0.05       0.03      0.14      0.11     0.22          0.13               0.18    0.08       0.18                 0.97          0.03       0.17           0.17       0.15        0.60    0.04              0.04         0.15               0.09               0.45                  0.04     0.31                0.09           0.78                     0.34               0.13         0.16             0.29       0.95                   0.13         0.17             0.43         0.08               0.16              0.29             nan                   nan                     nan                  0.06                    0.04                        0.15             nan             nan              0.11              nan               0.03             0.20            0.03          0.08         0.14            0.16          0.09      0.41             nan         0.06                0.15                   0.15                    nan        0.00             0.10           0.05   
3  

In [124]:
ratio_5_8 = d_metric_after_med.values / d_metric_before_med.values
ratio_5_8 = pd.DataFrame(ratio_5_8, columns=d_metric_after_med.columns)

In [125]:
ratio_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.66             0.67       1.98      1.36      2.33     0.93          1.14               0.87    1.09       0.50                 0.66          0.41       0.60           0.68       1.26        0.73    4.97              0.00         0.86               1.60               0.61                  0.96     0.77                0.80           0.69                     0.79               1.60         0.08             0.66       0.65                   1.10         0.68             0.51         2.72               1.09              0.87             nan                  1.64                    1.19                  2.72                    0.47                        0.45             nan             nan              0.27             1.55               0.11             0.64            0.19          1.14         0.41            1.70          1.43      0.89            1.35         0.00                0.60                   2.01                   2.80        0.64             0.40           1.08   
1      0.29             1.20       0.63      0.02      1.90     0.82          1.14               0.83    0.85       0.48                 0.71          0.30       0.55           0.85       1.65        0.80    2.12              0.11         1.17               1.13               0.60                  0.73     0.73                1.06           0.72                     0.86               1.76         0.01             0.89       0.63                   0.80         0.77             0.59         1.69               1.12              0.70            1.63                  1.16                    0.84                  1.92                    0.43                        0.55             nan             nan              0.50             1.09               0.17             0.35            0.08          0.86         0.18            1.93          1.64      0.76            1.06         0.28                0.68                   1.45                   1.40        0.70             0.61           1.92   
2      0.85             0.27       1.49      3.48      1.13     0.58          1.14               0.95    0.52       0.67                 0.65          0.19       0.50           0.91       1.26        0.74    4.97              0.97         0.74               1.60               0.88                  0.41     0.91                1.13           0.66                     0.84               3.15         0.75             1.07       0.73                   1.10         0.82             0.80         2.72               0.87              0.87             nan                   nan                     nan                  2.72                    0.47                        0.92             nan             nan              0.74              nan               0.25             0.74            0.31          0.92         0.78            2.00          1.01      0.96             nan         0.50                1.05                   1.64                    nan        0.03             1.95           0.70   
3  

In [126]:
mask_5_8 = ratio_5_8[ratio_5_8<=1.0]

In [127]:
mask_5_8

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.66             0.67        nan       nan       nan     0.93           nan               0.87     nan       0.50                 0.66          0.41       0.60           0.68        nan        0.73     nan              0.00         0.86                nan               0.61                  0.96     0.77                0.80           0.69                     0.79                nan         0.08             0.66       0.65                    nan         0.68             0.51          nan                nan              0.87             nan                   nan                     nan                   nan                    0.47                        0.45             nan             nan              0.27              nan               0.11             0.64            0.19           nan         0.41             nan           nan      0.89             nan         0.00                0.60                    nan                    nan        0.64             0.40            nan   
1      0.29              nan       0.63      0.02       nan     0.82           nan               0.83    0.85       0.48                 0.71          0.30       0.55           0.85        nan        0.80     nan              0.11          nan                nan               0.60                  0.73     0.73                 nan           0.72                     0.86                nan         0.01             0.89       0.63                   0.80         0.77             0.59          nan                nan              0.70             nan                   nan                    0.84                   nan                    0.43                        0.55             nan             nan              0.50              nan               0.17             0.35            0.08          0.86         0.18             nan           nan      0.76             nan         0.28                0.68                    nan                    nan        0.70             0.61            nan   
2      0.85             0.27        nan       nan       nan     0.58           nan               0.95    0.52       0.67                 0.65          0.19       0.50           0.91        nan        0.74     nan              0.97         0.74                nan               0.88                  0.41     0.91                 nan           0.66                     0.84                nan         0.75              nan       0.73                    nan         0.82             0.80          nan               0.87              0.87             nan                   nan                     nan                   nan                    0.47                        0.92             nan             nan              0.74              nan               0.25             0.74            0.31          0.92         0.78             nan           nan      0.96             nan         0.50                 nan                    nan                    nan        0.03              nan           0.70   
3  

features that got matched well for all combinations - standardarized absolute mean difference got reduced after matching ~ ratio <=1.0

In [128]:
mask_5_8.columns[~mask_5_8.isnull().any()].size

In [129]:
mask_5_8.columns[~mask_5_8.isnull().any()]

Index(['MBR_GNDR', 'CC_ADHD', 'CC_ALZHEIMER_PLUS', 'CC_ANEMIA', 'CC_ANXIETY_DISORDER', 'CC_ARTHRITIS', 'CC_ASTHMA', 'CC_BIPOLAR', 'CC_BREAST_CANCER', 'CC_CHRONIC_KIDNEY', 'CC_COLORECTAL_CANCER', 'CC_COPD', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS', 'CC_DIABETES', 'CC_DX_CNT', 'CC_EPILEPSY', 'CC_FIBROMYALGIA', 'CC_HEART_FAILURE', 'CC_HEPATITIS_C_CHRONIC', 'CC_HEPATITIS_C_UNSPECIFIED', 'CC_HEPATITIS_GEN', 'CC_HYPERLIPIDEMIA', 'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_ISCHEMIC', 'CC_LIVER', 'CC_MIGRAINE', 'CC_OBESITY', 'CC_PERSONALITY', 'CC_PTSD', 'CC_SCHIZOPHRENIA', 'CC_SCHIZOPHRENIA_OTHER', 'CC_TOBACCO', 'IP_BIN', 'ACUTE', 'IP_C', 'ER_C', 'ACUTE_C', 'IP_BIN6', 'ER_BIN6', 'ACUTE6', 'IP_BIN2', 'ACUTE2'], dtype='object')

features that didn't match for any combinations - standardarized absolute mean difference didn't got reduced after matching ~ ratio >=1.0

In [130]:
mask_5_8.columns[mask_5_8.isnull().all()].size

In [131]:
mask_5_8.columns[mask_5_8.isnull().all()]

Index(['ALLOW_IP', 'CC_AUTISM', 'CC_BPH', 'CC_CEREBRAL_PALSY', 'CC_DEVELOP_DELAYS', 'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E', 'CC_LEARN_DISAB', 'CC_MULTIPLE_SCLEROSIS', 'CC_MUSCULAR_DYSTROPHY', 'CC_VISUAL_IMPAIR', 'LOB', 'PREGNANCY'], dtype='object')

### 1.3 Merge and Save ratio

In [132]:
final_ratio = pd.concat([ratio_1_4, ratio_5_8], axis=0, ignore_index=True)
final_ratio

MBR_GNDR  AGE_AT_MIDMONTH  ALLOW_AMT  ALLOW_ER  ALLOW_IP  CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR  CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER  CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  CC_OSTEOPOROSIS  CC_PERIPHERAL  \
0      0.02             0.23       1.04      0.36      5.58     0.80          3.68               2.21    1.35       0.78                 0.69          6.13       0.70          13.42       0.96        0.78    0.12              0.00         0.55               2.03               0.94                  1.11     1.31                1.18           0.69                     1.00               1.34         1.07             1.00       0.99                   1.38         0.94             0.80         0.52               0.82              1.45             nan                  2.25                    1.63                  4.36                    0.93                        1.01             nan             nan              0.91              nan               0.14            11.57            9.27          0.66         2.51            1.13          0.50      1.11             nan         0.17                1.30                   1.49                   3.29        0.14             0.66           0.09   
1      0.08             0.38       0.65      0.89      6.17     0.71          3.68               1.88    1.15       0.85                 0.69          3.61       0.75           8.41       1.01        0.80    0.39              0.12         0.56               1.24               0.92                  1.22     1.15                0.83           0.74                     0.86               1.15         0.80             0.94       0.94                   1.01         0.84             0.75         0.46               0.77              1.31             nan                  1.12                    0.82                  2.18                    0.93                        0.84             nan             nan              0.88              nan               0.02             7.82            0.18          0.69         1.47            1.04          0.84      0.95            1.06         0.10                0.95                   1.72                   2.01        0.51             0.25           0.41   
2      0.16             0.16       0.55      1.38      2.67     0.72          2.69               1.88    0.68       0.99                 0.71         12.87       0.66           9.98       0.81        0.78    0.32              0.86         0.41               2.03               1.08                  1.11     1.34                0.83           0.71                     0.90               1.34         1.66             1.20       1.00                   1.01         0.77             0.95         0.70               0.58              1.33             nan                   nan                     nan                   nan                    0.79                        1.01             nan             nan              0.99              nan               0.09             7.23            3.48          0.79         2.62            1.13          1.01      1.05            0.67         0.19                1.17                   0.61                    nan        0.04             0.20           0.38   
3  

In [133]:
final_ratio.to_csv("./results/ratio/ratio_with_std_9_16.csv", index=False, header=True)